In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Polygon Meshes
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P28.jpg" style="width: 200px; display: inline;">


## Elements of a Mesh

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P12.jpg" style="width: 600px; display: inline;">


## Mesh Objects in Decod.es

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/3.00.D68 Mesh Large.jpg" style="width: 800px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P13.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P14.jpg" style="width: 200px; display: inline;">


<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P15.jpg" style="width: 200px; display: inline;">


### Mesh Usage

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.07.P21.jpg" style="width: 200px; display: inline;">
